# Collaborative Filtering 

### For movie recommenation

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql.types import StructField, StructType

### 1. Cargamos los DF

In [2]:
spark = SparkSession.builder.\
    master("local").\
        appName("collabFilter").\
            getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/04/01 16:33:15 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/04/01 16:33:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/01 16:33:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/01 16:33:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [37]:
df_movie = spark.read.options(header='True', inferSchema='True').csv("/mnt/d/Proyectos/Tutorial-SparkAWS/collaborative_data/movies.csv")
df_rating = spark.read.options(header='True', inferSchema='True').csv("/mnt/d/Proyectos/Tutorial-SparkAWS/collaborative_data/ratings.csv")

In [38]:
df_movie.show(2)
df_rating.show(2)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



In [39]:
df_rating.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



### 2. Joining DataFrames 

In [40]:
df_rating.join(df_movie, on='movieId', how = 'inner').show(3)

+-------+------+------+---------+--------------------+--------------------+
|movieId|userId|rating|timestamp|               title|              genres|
+-------+------+------+---------+--------------------+--------------------+
|      1|     1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|
|      3|     1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|
|      6|     1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|
+-------+------+------+---------+--------------------+--------------------+
only showing top 3 rows



In [41]:
df_final = df_rating.join(df_movie, on = "movieId", how = "inner")
df_final.show(4)

+-------+------+------+---------+--------------------+--------------------+
|movieId|userId|rating|timestamp|               title|              genres|
+-------+------+------+---------+--------------------+--------------------+
|      1|     1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|
|      3|     1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|
|      6|     1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|
|     47|     1|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|
+-------+------+------+---------+--------------------+--------------------+
only showing top 4 rows



__Controlamos que no haya duplicados__

In [42]:
df_final.groupBy(col("movieId"), col("userId")).\
    agg(count(col("movieId")).alias('miCount')).\
        filter(col('miCount')> 1).\
            show()

+-------+------+-------+
|movieId|userId|miCount|
+-------+------+-------+
+-------+------+-------+



+ Hacemos Split de los datos 80/20

In [46]:
df_train, df_test = df_final.randomSplit([.8, .2])

In [48]:
df_train.show(3)
df_train.count()

+-------+------+------+----------+----------------+--------------------+
|movieId|userId|rating| timestamp|           title|              genres|
+-------+------+------+----------+----------------+--------------------+
|      1|     5|   4.0| 847434962|Toy Story (1995)|Adventure|Animati...|
|      1|     7|   4.5|1106635946|Toy Story (1995)|Adventure|Animati...|
|      1|    15|   2.5|1510577970|Toy Story (1995)|Adventure|Animati...|
+-------+------+------+----------+----------------+--------------------+
only showing top 3 rows



80749

### 3. Creacion de modelo ASL

In [50]:
from pyspark.ml.recommendation import ALS

In [54]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", 
nonnegative=True, implicitPrefs=False, coldStartStrategy="drop")

### 3.1 Tuning del modelo

In [73]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

__3.1.1 creamos el grid con 16 modelos posibles__

In [76]:
param_grid = ParamGridBuilder()\
    .addGrid(als.rank, [10, 50, 100, 150])\
    .addGrid(als.regParam, [.01, .05, .1, .15])\
    .build()

__3.1.2 creamos un evaluador__

In [77]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
                                )

__3.1. creamos un cross validator que definirá el mejor modelo__

In [78]:
cv = CrossValidator(
                    estimator=als,
                    estimatorParamMaps=param_grid,
                    evaluator=evaluator,
                    numFolds=5
                    )

__3.1.4 Entrenamos el modelo__

In [59]:
model = cv.fit(df_train)

24/04/01 17:58:42 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/01 17:58:42 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


Exception ignored in: <function JavaWrapper.__del__ at 0x7fb265c62b90>          
Traceback (most recent call last):
  File "/home/nicoudin/spark/spark-3.3.2-bin-hadoop3/python/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'ALS' object has no attribute '_java_obj'


__3.1.5 Evaluamos el modelo y buscamos el mejor__

In [84]:
best_model = model.bestModel
testPrediction = best_model.transform(df_test)
RMSE = evaluator(testPrediction)


TypeError: 'RegressionEvaluator' object is not callable

El mejor modelo que obtuvimos es de: print(RMSE)

__3.1.6 Hacemos recomenaciones__

Usando .recommended(n) lo que obtenemos es el top N de las recomenaciones.